# cifal-20

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Bidirectional, Flatten, Dense, SimpleRNN, LSTM, BatchNormalization, Dropout, Activation
from tensorflow.keras.initializers import RandomNormal, Constant


In [ ]:
# 데이터 Load
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar100.load_data(label_mode="coarse") # fine 은 cifal-100 dataset, coarse는 cifal-20 dataset
train_images = train_images.reshape((50000,32,32,3))
test_images = test_images.reshape((10000,32,32,3))
train_images, test_images = train_images / 255.0, test_images / 255.0 # 정규화


169001437/169001437 [==============================] - 13s 0us/step


In [ ]:
# 원-핫 인코딩
one_hot_train_labels = to_categorical(train_labels, 20)
one_hot_test_labels = to_categorical(test_labels, 20)

In [ ]:
# 모델 학습
# 컨볼루션 - 배치정규화 - 활성화함수 - 맥스풀링 - 드롭아웃 순으로 적용
# 2차원 배열로 바꾸고 GRU로 학습
# Flastten()으로 1차원 배열로 바꾼 후 출력

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same', strides=(1, 1), input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(tf.keras.layers.Reshape(target_shape = (64, 16*16)))
model.add(Bidirectional(tf.keras.layers.GRU(64, input_shape = (64, 16*16), return_sequences = True)))

model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization(momentum=0.95, epsilon=0.005, beta_initializer=RandomNormal(mean=0.0, stddev=0.05), gamma_initializer=Constant(value=0.9)))


model.add(tf.keras.layers.Flatten()) 
model.add(Dense(20, activation='softmax'))

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(train_images, one_hot_train_labels, epochs=8, batch_size=128,  validation_data=(test_images, one_hot_test_labels), validation_split=0.3)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        3

In [ ]:
print("\n=============test results==========")
labels=model.predict(test_images)
print("\n Accuracy: %.4f" % (model.evaluate(test_images, one_hot_test_labels)[1]))
print("===================================")


=============test results==========
313/313 [==============================] - 2s 6ms/step - loss: 1.4847 - accuracy: 0.5366

 Accuracy: 0.5366
